 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 5: Optimisation avec contraintes et calcul variationnel
Tangi Migot

In [1]:
using Krylov, LinearAlgebra, Logging, NLPModels, NLPModelsIpopt, Printf, SolverCore, Test

In [4]:
using PDENLPModels, Gridap

## Quelques commentaires en Julia

### Les kwargs: choix optionnels

Dans le projet du dernier labo, une des questions demandait d'ajouter une option pour utiliser la fonction `lsmr` ou `lsqr`. C'est le cas typique d'arguments optionnels:
- On veut proposer un choix par défaut à l'utilisateur, par exemple `lsqr`;
- On veut laisser la possibilité à l'utilisateur de changer;
- On voudrait aussi pouvoir ajouter d'autres par la suite (sans avoir à tout modifier).

In [9]:
function dsol(A, b, ϵ; solver :: Function = lsqr)
    (d, stats) = solver(A, b, atol = ϵ)
    return d
end

dsol (generic function with 1 method)

A noter que l'on donne des valeurs par défaut aux arguments qui apparaissent après le `;`.

## Exercice 1: Pénalité quadratique pour les ADNLPModels

Dans cet exercice, on va étudier une version simple d'une méthode de pénalité quadratique pour les problèmes d'optimisation avec contraintes d'égalité.
```math
min f(x) s.à c(x) = 0.
```
Dans les labos précédents, on a déjà utilisé un NLPModel particulier, le ADNLPModel:

In [118]:
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau = ADNLPModel(fH, x0H, cH, [0.], [0.])

ADNLPModel - Model with automatic differentiation backend ADNLPModels.ForwardDiffAD{ForwardDiff.GradientConfig{ForwardDiff.Tag{typeof(fH), Float64}, Float64, 2, Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(fH), Float64}, Float64, 2}}}}(3, 2, ForwardDiff.GradientConfig{ForwardDiff.Tag{typeof(fH), Float64}, Float64, 2, Vector{ForwardDiff.Dual{ForwardDiff.Tag{typeof(fH), Float64}, Float64, 2}}}((Partials(1.0, 0.0), Partials(0.0, 1.0)), ForwardDiff.Dual{ForwardDiff.Tag{typeof(fH), Float64}, Float64, 2}[Dual{ForwardDiff.Tag{typeof(fH), Float64}}(0.0,6.95176457205923e-310,6.9517648938691e-310), Dual{ForwardDiff.Tag{typeof(fH), Float64}}(0.0,6.95176457205923e-310,6.951764570645e-310)]))
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ████████████████████ 1     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           u

Attention: dans toute la suite de l'exercice on suppose que les bornes sur les contraintes `nlp.meta.lcon` et `nlp.meta.ucon` sont 0 pour simplifier.

### Question 1: Transformer un ADNLPModel en un problème pénalisé
Coder la fonction `quad_penalty_adnlp` qui prend en entrée un ADNLPModel, et un paramètre ρ et qui retourne un nouveau ADNLPModel qui correspond au problème sans contrainte:
$$
\min_x f(x) + \frac{\rho}{2}\|c(x)\|^2.
$$
Remarque: on peut accèder aux fonctions f et c par `nlp.f` et `nlp.c`.

In [68]:
function quad_penalty_adnlp(nlp :: ADNLPModel, ρ :: Real)
    # TODO
    f_quad(x) = nlp.f(x) + (ρ / 2) * nlp.c(x)' * nlp.c(x)
    # f_quad(x) = nlp.f(x) + (ρ / 2) * (norm(nlp.c(x))^2)
    x0_quad = nlp.meta.x0
    nlp_quad = ADNLPModel(f_quad, x0_quad)
    return nlp_quad
end

quad_penalty_adnlp (generic function with 1 method)

In [69]:
#Faire des tests pour vérifier que ça fonctionne.
fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
x0H = [10., 20.]
himmelblau = ADNLPModel(fH, x0H)

himmelblau_quad = quad_penalty_adnlp(himmelblau, 1)
@test himmelblau_quad.meta.ncon == 0
@test obj(himmelblau_quad, zeros(2)) == 170

Test Passed

In [77]:
#Ajouter au moins un autre test similaire avec des contraintes.
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau_1 = ADNLPModel(fH, x0H, cH, [0.], [0.])

himmelblau_1_quad = quad_penalty_adnlp(himmelblau_1, 1)
@test himmelblau_1_quad.meta.ncon == 0
@test obj(himmelblau_1_quad, zeros(2)) == 170.5
@test obj(himmelblau_1_quad, ones(2)) == fH([1., 1.]) + cH([1.])[1] / 2

106.0

Test Passed

In [78]:
# Ajouter un test au cas ou `nlp.meta.lcon` ou `nlp.meta.ucon` ont des composantes differentes de 0.
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau_2 = ADNLPModel(fH, x0H, cH, [1.], [2.])

himmelblau_2_quad = quad_penalty_adnlp(himmelblau_2, 1)
@test himmelblau_2_quad.meta.ncon == 0
@test obj(himmelblau_2_quad, zeros(2)) == 170.5
@test obj(himmelblau_2_quad, ones(2)) == fH([1., 1.]) + cH([1.])[1] / 2

Test Passed

### Question 2: KKT
Coder une fonction `KKT_eq_constraint(nlp :: AbstractNLPModel, x, λ)` qui vérifie si le point `x` avec multiplicateur de Lagrange `λ` satisfait les conditions KKT d'un problème avec contraintes d'égalités.

In [161]:
function KKT_eq_constraint(nlp :: AbstractNLPModel, x, λ)
   # TODO
   KKT = grad(nlp, x) + jac(nlp, x)' * λ
   ϵ = 1e-3
   if norm(KKT) <= ϵ && norm(nlp.c(x)) <= ϵ
      return true
   else
      return false
   end
end

KKT_eq_constraint (generic function with 1 method)

In [145]:
#test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau_3 = ADNLPModel(fH, x0H, cH, [0.], [0.])

@test KKT_eq_constraint(himmelblau_3, zeros(2), 1) == false

2×1 Matrix{Float64}:
 -13.0
 -22.0

Test Passed

### Question 3: méthode de pénalité quadratique

In [59]:
using NLPModelsIpopt

In [146]:
function quad_penalty(nlp      :: ADNLPModel, #Note que ça ne marche pas pour tous les NLPModel! 
                      x        :: AbstractVector; 
                      ϵ        :: AbstractFloat = 1e-3,
                      η        :: AbstractFloat = 1e6, 
                      σ        :: AbstractFloat = 2.0,
                      max_eval :: Int = 1_000, 
                      max_time :: AbstractFloat = 60.,
                      max_iter :: Int = typemax(Int64)
                      )
    ##### Initialiser cx et gx au point x;
    # cx = nlp.c(x)       # Initialiser la violation des contraintes
    cx = nlp.c(x)
    gx = grad(nlp, x)   # Initialiser le gradient
    ######################################################
    normcx = normcx_old = norm(cx)

    ρ = 1.

    iter = 0    

    el_time = 0.0
    tired   = neval_cons(nlp) > max_eval || el_time > max_time
    status  = :unknown

    start_time = time()
    too_small  = false
    normdual   = norm(gx) #exceptionnellement on ne va pas vérifier toute l'optimalité au début.
    optimal    = max(normcx, normdual) ≤ ϵ
    
    nlp_quad   = quad_penalty_adnlp(nlp, ρ)

    @info log_header([:iter, :nf, :primal, :status, :nd, :Δ],
    [Int, Int, Float64, String, Float64, Float64],
    hdr_override=Dict(:nf => "#F", :primal => "‖F(x)‖", :nd => "‖d‖"))

    while !(optimal || tired || too_small)

        #Appeler Ipopt pour résoudre le problème pénalisé en partant du point x0 = x.
        #utiliser l'option print_level = 0 pour enlever les affichages d'ipopt.
        stats = ipopt(nlp_quad, x0 = x, print_level = 0)
        ################################################
      
        if stats.status == :first_order
            ###### Mettre à jour cx avec la solution renvoyé par Ipopt
            x = stats.solution
            cx = nlp.c(x)
            ##########################################################
            normcx_old = normcx
            normcx = norm(cx)
        end
        
        if normcx_old > 0.95 * normcx
            ρ *= σ
        end

        @info log_row(Any[iter, neval_cons(nlp), normcx, stats.status])
        
        nlp_quad   = quad_penalty_adnlp(nlp, ρ)

        el_time      = time() - start_time
        iter   += 1
        many_evals   = neval_cons(nlp) > max_eval
        iter_limit   = iter > max_iter
        tired        = many_evals || el_time > max_time || iter_limit || ρ ≥ η
        ##### Utiliser la réalisabilité dual renvoyé par Ipopt pour `normdual`
        normdual     = stats.dual_feas
        ###################################################################
        optimal      = max(normcx, normdual) ≤ ϵ
    end

    status = if optimal 
        :first_order
    elseif tired
        if neval_cons(nlp) > max_eval
            :max_eval
        elseif el_time > max_time
            :max_time
        elseif iter > max_iter
            :max_iter
        else
            :unknown_tired
        end
    elseif too_small
        :stalled
    else
        :unknown
    end

    return GenericExecutionStats(nlp, status = status, solution = x,
                                 objective = obj(nlp, x),
                                 primal_feas = normcx,
                                 dual_feas = normdual,
                                 iter = iter, 
                                 elapsed_time = el_time,
                                 solver_specific = Dict(:penalty => ρ))
end

quad_penalty (generic function with 1 method)

In [155]:
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau = ADNLPModel(fH, x0H, cH, [0.], [0.])

#Faire des tests pour vérifier que ça fonctionne.
stats = quad_penalty(himmelblau, 2ones(2))
@test stats.status == :first_order
@test stats.solution ≈ [1.0008083416169895, 2.709969135758311]
@test norm(cons(himmelblau, stats.solution)) ≈ 0. atol=1e-3

┌ Info:   iter      #F    ‖F(x)‖           status       ‖d‖         Δ  
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:32
┌ Info:      0       0   2.0e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      1       0   2.0e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      2       0   1.9e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56


┌ Info:      3       0   1.9e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      4       0   1.7e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      5       0   1.5e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      6       0   1.1e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      7       0   5.5e-01      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      8       0   2.4e-01      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      9       0   1.1e-01      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:     10       0   5.4e-02      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:     11       0   2.6e-02      fi

Test Passed

Vérifier que la solution rendue vérifie les conditions KKT avec la fonction de la question précédente.

In [162]:
# TODO
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau = ADNLPModel(fH, x0H, cH, [0.], [0.])

stats = quad_penalty(himmelblau, 2ones(2))
@test KKT_eq_constraint(himmelblau, stats.solution, - Matrix(jac(himmelblau, stats.solution)') \ grad(himmelblau, stats.solution)) == true

┌ Info:   iter      #F    ‖F(x)‖           status       ‖d‖         Δ  
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:32


┌ Info:      0       0   2.0e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      1       0   2.0e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      2       0   1.9e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      3       0   1.9e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      4       0   1.7e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      5       0   1.5e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      6       0   1.1e+00      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      7       0   5.5e-01      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:      8       0   2.4e-01      fi

┌ Info:     14       0   3.2e-03      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:     15       0   1.6e-03      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56
┌ Info:     16       0   8.1e-04      first_order
└ @ Main c:\Users\Goglu\GitClones\MTH8408\lab5\Lab5-notebook.ipynb:56


Test Passed

In [163]:
#Fichier de tests à demander.
tol = 1e-3
@testset "Simple problem" begin
    n = 10
    nlp = ADNLPModel(x->dot(x, x), zeros(n),
                     x->[sum(x) - 1], zeros(1), zeros(1))

    stats = with_logger(NullLogger()) do
      quad_penalty(nlp, nlp.meta.x0, ϵ=1e-6)
    end
    dual, primal, status = stats.dual_feas, stats.primal_feas, stats.status
    @test norm(n * stats.solution - ones(n)) < tol
    @test dual < tol
    @test primal < tol
    @test status == :first_order
end

@testset "Rosenbrock with ∑x = 1" begin
    nlp = ADNLPModel(x->(x[1] - 1.0)^2 + 100 * (x[2] - x[1]^2)^2, 
                     [-1.2; 1.0],
                     x->[sum(x)-1], [0.0], [0.0])

    stats = with_logger(NullLogger()) do
      quad_penalty(nlp, nlp.meta.x0)
    end
    dual, primal, status = stats.dual_feas, stats.primal_feas, stats.status
    @test dual < tol#1e-6
    @test primal < tol
    @test status == :first_order
end

@testset "HS6" begin
    nlp = ADNLPModel(x->(1 - x[1])^2, [-1.2; 1.0],
                     x->[10 * (x[2] - x[1]^2)], [0.0], [0.0])

    stats = with_logger(NullLogger()) do
      quad_penalty(nlp, nlp.meta.x0)
    end
    dual, primal, status = stats.dual_feas, stats.primal_feas, stats.status
    @test dual < tol
    @test primal < tol
    @test status == :first_order
end

@testset "HS7" begin
    nlp = ADNLPModel(x->log(1 + x[1]^2) - x[2], 
                     [2.0; 2.0],
                     x->[(1 + x[1]^2)^2 + x[2]^2 - 4], [0.0], [0.0])

    stats = with_logger(NullLogger()) do
      quad_penalty(nlp, nlp.meta.x0)
    end
    dual, primal, status = stats.dual_feas, stats.primal_feas, stats.status
    @test dual < tol
    @test primal < tol
    @test status == :first_order
end

Test Summary:  | Pass  Total  Time
Simple problem |    4      4  4.9s


Test Summary:          | Pass  Total  Time
Rosenbrock with ∑x = 1 |    3      3  0.7s


Test Summary: | Pass  Total  Time
HS6           |    3      3  0.6s


Test Summary: | Pass  Total  Time
HS7           |    3      3  0.6s


Test.DefaultTestSet("HS7", Any[], 3, false, false, true, 1.678723159607e9, 1.678723160223e9)

## Exercice 2: Calcul Variationnel

Dans cet exercice, on considère le problème de calcul variationnel suivant:
$$
\min \int_0^1 (\dot{x}(t)^2+2x(t)^2)e^t dt, \quad x(0)=0, x(1)=e - e^{-2}
$$

modélisé avec `PDENLPModels`.

In [164]:
function cv_model(n :: Int)

  domain = (0,1) # set the domain
  partition = n
  model = CartesianDiscreteModel(domain,partition) # set discretization
    
  labels = get_face_labeling(model)
  add_tag_from_tags!(labels,"diri1",[2])
  add_tag_from_tags!(labels,"diri0",[1]) # boundary conditions

  order=1
  valuetype=Float64
  reffe = ReferenceFE(lagrangian, valuetype, order)
  V0 = TestFESpace(model, reffe; conformity=:H1, dirichlet_tags=["diri0","diri1"])
  U = TrialFESpace(V0,[0., exp(1)-exp(-2)])

  trian = Triangulation(model)
  degree = 2
  dΩ = Measure(trian,degree) # integration machinery

  # Our objective function
  w(x) = exp(x[1])
  function f(y)
    ∫((∇(y)⊙∇(y) + 2 * y * y) * w) * dΩ
  end

  xin = zeros(Gridap.FESpaces.num_free_dofs(U))
  nlp = GridapPDENLPModel(xin, f, trian, U, V0)
  return nlp
end

cv_model (generic function with 1 method)

### Question 1: Résoudre
Résoudre le NLPModel généré par la fonction `cv_model` pour `n = 16` avec `ipopt` et afficher la solution (attention la solution rendue ne contient pas les valeurs aux bords qu'il faut rajouter).

In [165]:
# TODO
stats = ipopt(cv_model(16))
x16 = vcat([0], stats.solution, [exp(1) - exp(-2)])

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       44

Total number of variables............................:       15
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.8202747e+02 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

"Execution stats: first-order stationary"

### Question 2: Convergence en `n`
Afficher sur un même graphique la solution obtenue par `ipopt` pour plusieurs valeurs de `n`.

In [169]:
using Plots

stats8 = ipopt(cv_model(8))
x8 = vcat([0], stats8.solution, [exp(1) - exp(-2)])

stats16 = ipopt(cv_model(16))
x16 = vcat([0], stats16.solution, [exp(1) - exp(-2)])

stats32 = ipopt(cv_model(32))
x32 = vcat([0], stats32.solution, [exp(1) - exp(-2)])

plot(x8, x16, x32)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.3784624e+02 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

### Question 3: Comparer à la solution exacte

La solution exacte est $x(t)=e^t - e^{-2t}$ et la valeur optimale est $e^3 - 2e^{-3}+1$.